In [4]:
import pandas as pd
import MySQLdb as db
from datetime import datetime
import os

In [2]:
# DATETIME format: YYYY-MM-DD HH:MI:SS.

# table class 
class data():
    def __init__(self, name):
        # Connect to Database
        self.con = db.connect(host="localhost", user="root", passwd='', db=name)
        self.cursor = self.con.cursor()
    
    def createTable(self, tname):
        # Create a table
        sql = "CREATE TABLE " + tname + " (DateTime datetime, Room VARCHAR(20), Associated int, Authenticated int, PRIMARY KEY(DateTime, Room))"
        self.cursor.execute(sql)
        
    def openFile(self, tname, fname):
        # Find keyworrd 'key' in csv file and read into data frame from there
        with open(fname) as f:
            # f is the file name, starting on row 1
            # Accounts for csvs where 'key' is in a different row
            # num = counter
            for num, line in enumerate(f,1):
                if line.startswith('Key'):
                    break
        df = pd.read_csv(fname, skiprows=num-1)
        return df
    
    def fixFile(self, df):
        for i in range(0, len(df)):
            # put time into sql format
            df['Event Time'][i] = df['Event Time'][i].replace('GMT+00:00','')
            df['Event Time'][i] = datetime.strptime(df['Event Time'][i], '%a %b %d %X %Y')
            # Split column Key (contains campus, building and room) into separate parts so they can be added to separate columns of database table
            df['Key'][i] = df['Key'][i].split(' > ')
        return df
        
#     def addFile(self, tname, df):
#         # put data from dataframe into table in database
#         for i in range(0, len(df)):
#             self.cursor.execute("INSERT INTO " + tname + "(Campus, Building, Room, DateTime, Associated, Authenticated) VALUES('%s', '%s', '%s', '%s', %d, %d)" 
#                                 % (df['Key'][i][0], df['Key'][i][1], df['Key'][i][2], df['Event Time'][i], df['Associated Client Count'][i], 
#                                  df['Authenticated Client Count'][i]))
#             self.con.commit()           
#         return

    def addFile(self, tname, df):
        # put data from dataframe into table in database
        for i in range(0, len(df)):
            self.cursor.execute("INSERT INTO " + tname + "(Room, DateTime, Associated, Authenticated) VALUES('%s', '%s', %d, %d)" 
                                % (df['Key'][i][2], df['Event Time'][i], df['Associated Client Count'][i], 
                                 df['Authenticated Client Count'][i]))
            self.con.commit()           
        return


name = 'DatabaseMain'
myData = data(name)
tname = 'ExtraWiFiLogData'
myData.createTable(tname)

path = '/Users/JoanMcCarthy/Google Drive/Computer Science/Research Practicum/Data/CSI WifiLogsExtra'

for fn in os.listdir(path):
    if fn.lower().endswith('.csv'):
        #print "File Name: " + fn
        fname = os.path.join(path, fn)
        df = myData.openFile(tname, fname)
        df = myData.fixFile(df)
        myData.addFile(tname, df)

/Users/JoanMcCarthy/anaconda/envs/proj2/lib/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JoanMcCarthy/anaconda/envs/proj2/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JoanMcCarthy/anaconda/envs/proj2/lib/python2.7/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# CREATE A TABLE WITH AVERAGE NUM OF WIFI CONNECTIONS BETWEEN QUARTER PAST AND QUARTER TO THE HOUR IN A ROOM

# connect to database
con = db.connect(host="localhost", user="root", passwd='', db='DatabaseMain')
cursor = con.cursor()

In [3]:


# create table
tname = 'ExtraAverageNumWifiConnections'
sql = "CREATE TABLE IF NOT EXISTS " + tname + " (DateTime datetime, Room VARCHAR(10), AvgNumWifiConn FLOAT, PRIMARY KEY (DateTime, Room))"
cursor.execute(sql)

# Dates and Times to be queried
Date = ['2015-11-03', '2015-11-04', '2015-11-05', '2015-11-06', '2015-11-09', '2015-11-10', '2015-11-11', '2015-11-12', '2015-11-13', '2015-11-16']
Time = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

d = "2016-07-18"

for t in Time:
    # set start date time and end datetime to query between
    start = "'" + d + " " + t + ":15:00'"
    end = "'" + d + " " + t + ":45:00'"
    # set datetime to enter into database (start of the hour being queried)
    datetime = d + " " + t + ":00:00"
    # Query B-004 between set datetime
    sql1 = "SELECT AVG(Associated) FROM ExtraWiFiLogData WHERE DateTime BETWEEN " + start + " AND " + end + " AND Room = 'B-004'"
    cursor.execute(sql1)
    res1 = cursor.fetchall()
    # insert result into table
    cursor.execute("INSERT INTO " + tname + " (DateTime, Room, AvgNumWifiConn) VALUES ('%s', '%s', %f)" % (datetime, 'B-004',res1[0][0] if res1[0][0] else 0))
    con.commit() 
    # Query B-003 between set datetime 
    sql2 = "SELECT AVG(Associated) FROM ExtraWiFiLogData WHERE DateTime BETWEEN " + start + " AND " + end + " AND Room = 'B-003'"
    cursor.execute(sql2)
    res2 = cursor.fetchall()
    # insert result into table
    cursor.execute("INSERT INTO " + tname + " (DateTime, Room, AvgNumWifiConn) VALUES ('%s', '%s', %f)" % (datetime, 'B-003', res2[0][0] if res2[0][0] else 0))
    con.commit() 
    # Query B-002 between set datetime
    sql3 = "SELECT AVG(Associated) FROM ExtraWiFiLogData WHERE DateTime BETWEEN " + start + " AND " + end + " AND Room = 'B-002'"
    cursor.execute(sql3)
    res3 = cursor.fetchall()
    # insert result into table
    cursor.execute("INSERT INTO " + tname + " (DateTime, Room, AvgNumWifiConn) VALUES ('%s', '%s', %f)" % (datetime, 'B-002', res3[0][0] if res3[0][0] else 0))
    con.commit() 

/Users/JoanMcCarthy/anaconda/envs/proj2/lib/python2.7/site-packages/ipykernel/__main__.py:10: Warning: Table 'ExtraAverageNumWifiConnections' already exists


In [5]:
sql = """SELECT AvgNumWifiConn FROM ExtraAverageNumWifiConnections"""
df = pd.read_sql_query(sql, con)

In [8]:
df.to_csv('ExtraDataAvgs.csv', index=False)

In [9]:
# Create table
sql = """SELECT W.DateTime, W.Room, R.Capacity, W.Associated
            FROM DatabaseMain.ExtraWifiLogData W, DatabaseMain.Rooms R
            WHERE W.Room = R.Room """
df = pd.read_sql_query(sql, con)

In [12]:
df.to_csv('ExtraData.csv', index=False)